In [1]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [2]:
fake = faker.Faker(locale='en')

In [3]:
headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'referer': 'http://www.deepmind.com/careers'
}

In [4]:
URL = "http://api.greenhouse.io/v1/boards/deepmind/embed/jobs"

In [5]:
snapshot_pattern = re.compile(r"(\nSnapshot:?\n)((.+?)(\n))", flags=re.DOTALL|re.MULTILINE|re.IGNORECASE)
responsibilities_pattern = re.compile(r"(\nKey Responsibilities:?\n\n)((.+?)(\n\n))", flags=re.DOTALL|re.MULTILINE|re.IGNORECASE)
qualifications_pattern = re.compile(r"(\nAbout You:?\n)((.*)($))", flags=re.DOTALL|re.MULTILINE|re.IGNORECASE)

In [6]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

In [7]:
result = requests.get(URL, headers=headers)
result

<Response [200]>

In [8]:
all_jobs = result.json()['jobs']
len(all_jobs), all_jobs[1]

(47,
 {'absolute_url': 'https://boards.greenhouse.io/deepmind/jobs/4380039',
  'data_compliance': [{'type': 'gdpr',
    'requires_consent': False,
    'retention_period': None}],
  'internal_job_id': 2305052,
  'location': {'name': 'London, UK'},
  'metadata': [{'id': 90484,
    'name': 'Website Grouping',
    'value': 'Operations',
    'value_type': 'single_select'}],
  'id': 4380039,
  'updated_at': '2022-08-09T08:01:53-04:00',
  'requisition_id': None,
  'title': 'Assistant to Two leads, Operations'})

In [9]:
job_dicts = []
for job_info in tqdm(all_jobs):
    url = job_info['absolute_url']
    publish_date = parser.parse(job_info['updated_at'])
    job_website = requests.get(url, headers=headers)

    soup = BeautifulSoup(job_website.text)
    contents = soup.find('div', id='content')

    description = re.search(snapshot_pattern, contents.text)
    if description:
        description = clear_string(description.group(3))

    responsibilities = re.search(responsibilities_pattern, contents.text)
    if responsibilities:
        responsibilities = clear_string(responsibilities.group(3))

    qualifications = re.search(qualifications_pattern, contents.text)
    if qualifications:
        qualifications = clear_string(qualifications.group(3))

    if description is None and responsibilities is None and qualifications is None:
        print(f'All None for {url}. skipping...')
        continue

    job_dict = {
        'title': job_info['title'],
        'internal_id' : job_info['internal_job_id'],
        'url': url,
        'description': description,
        'responsibilities': responsibilities,
        'qualifications': qualifications,
        'company': 'DeepMind',
        'publish_date': publish_date
    }

    job_dicts.append(job_dict)

    time.sleep(0.2) # sleep for 200 ms

100%|██████████| 47/47 [00:37<00:00,  1.25it/s]


In [10]:
snapshot = pd.DataFrame(job_dicts)
snapshot.sample(5)

,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
26,Research Analytics Specialist,2333806,https://boards.greenhouse.io/deepmind/jobs/446...,The Research Insights & Systems Evolution (RIS...,None,What we are looking for in the ideal candidate...,DeepMind,2022-08-05 10:54:18-04:00
46,Workplace Manager,2163290,https://boards.greenhouse.io/deepmind/jobs/397...,"At DeepMind, we relate our success directly to...",Focussed on the delivery of day-to-day operati...,You will be experienced in operational deliver...,DeepMind,2022-08-05 10:08:56-04:00
3,Business & Corporate Development Manager,2328832,https://boards.greenhouse.io/deepmind/jobs/444...,The newly formed Business and Corporate Develo...,Develop a deep understanding of DeepMind’s res...,In order to set you up for success as a BCD Ma...,DeepMind,2022-08-08 09:32:41-04:00
5,Employee Relations Partner,1726743,https://boards.greenhouse.io/deepmind/jobs/292...,DeepMind has a remarkable track record of scie...,Crafting and delivering people practises which...,"You are driven by fairness, high quality deliv...",DeepMind,2022-08-05 10:08:56-04:00
17,"Principal, Strategy & Operations for Life Scie...",2308577,https://boards.greenhouse.io/deepmind/jobs/444...,Artificial Intelligence could be one of humani...,Manage consultancies: we plan to work with two...,In order to set you up for success as a Princi...,DeepMind,2022-08-05 10:08:56-04:00


In [13]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             47 non-null     object        
 1   internal_id       47 non-null     int64         
 2   url               47 non-null     object        
 3   description       44 non-null     object        
 4   responsibilities  32 non-null     object        
 5   qualifications    43 non-null     object        
 6   company           47 non-null     object        
 7   publish_date      47 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 3.1+ KB


In [12]:
snapshot['publish_date'] = snapshot['publish_date'].apply(lambda x: x.replace(tzinfo=None))
snapshot['publish_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 47 entries, 0 to 46
Series name: publish_date
Non-Null Count  Dtype         
--------------  -----         
47 non-null     datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 504.0 bytes


In [14]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'10-08-2022'

In [15]:
snapshot.to_csv(f'../data/deepmind/{current_date}.csv')
snapshot.to_excel(f'../data/deepmind/{current_date}.xlsx', engine='xlsxwriter')

### Проверка на одной вакансии

In [9]:
parser.parse(all_jobs[4]['updated_at'])

datetime.datetime(2022, 6, 20, 11, 5, 42, tzinfo=tzoffset(None, -14400))

In [10]:
job_website = requests.get(all_jobs[4]['absolute_url'], headers=headers)
job_website

<Response [200]>

In [11]:
soup = BeautifulSoup(job_website.text)

In [12]:
contents = soup.find('div', id='content')

In [13]:
contents.text

'\nAt DeepMind, we value diversity of experience, knowledge, backgrounds and perspectives and harness these qualities to create extraordinary impact. We are committed to equal employment opportunity regardless of sex, race, religion or belief, ethnic or national origin, disability, age, citizenship, marital, domestic or civil partnership status, sexual orientation, gender identity, pregnancy, maternity or related condition (including breastfeeding) or any other basis as protected by applicable law. If you have a disability or additional need that requires accommodation, please do not hesitate to let us know.\nSnapshot\nDeepMind has a remarkable track record of scientific breakthroughs. Such extraordinary work is a direct result of the brilliant and diverse people we bring together. Our People & Culture team work together to maintain, optimise, and nurture our culture to create the best environment in the world for advancing AI research. A place where everyone’s expertise is recognised 

In [14]:
description = re.search(snapshot_pattern, contents.text)
description.group(3)

'DeepMind has a remarkable track record of scientific breakthroughs. Such extraordinary work is a direct result of the brilliant and diverse people we bring together. Our People & Culture team work together to maintain, optimise, and nurture our culture to create the best environment in the world for advancing AI research. A place where everyone’s expertise is recognised and where everyone is continually learning and supported to be the best leaders, managers, and collaborators they can be.'

In [15]:
responsibilities = re.search(responsibilities_pattern, contents.text)
responsibilities.group(3)

'Crafting and delivering people practises which develop and maintain positive working relationships.\nSolving HR and policy issues, anything from handling long term sickness to working on a disciplinary investigation which includes advising on best practice and note-taking responsibilities, to assisting in the management of poor performance.\nSupporting large-scale change programmes, and being a point of contact for topics such as organisational restructures, maternity and paternity, sickness absence.\nHelping ensure our employee handbook, policies and code of conduct are up to date, legally compliant and reflect our culture and ways of working. .\nIdentifying and implementing appropriate reporting mechanisms for employee relations cases, both informal and formal.\nPartnering closely with Learning and People Partners to ensure managers are equipped to support with people challenges in a fair and effective way, and to enable you to build strong relationships across the organisation.\nYo

In [16]:
qualifications = re.search(qualifications_pattern, contents.text)
qualifications.group(3)

'\nYou are driven by fairness, high quality delivery and standards, and care deeply about people.\nWhen handling people issues, you efficiently navigate legal, cultural and business risks and tensions.\nYou enjoy conflict management and problem solving.\nYou have experience of handling investigation and disciplinary processes.\xa0\nYou enjoy learning about and working within non-UK legal jurisdictions.\nYou have change management experience.\nYou hold a solid understanding of organisational culture. Different work environments pose different challenges for people, so too does the ever-changing nature of our organisation as we grow, scale, and evolve.\nYou proactively ensure your Employment Law expertise is up-to-date.\nTaking the learning opportunities after each case you support, you possess a continuous improvement approach\n\n'